In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def FNS(scores):
    domination = (scores[:, None, :] <= scores[None, :, :]).all(2) # domination[i, j] = "i dominuje j"
    domination &= ~(scores[:, None, :] == scores[None, :, :]).all(2)
    Nx = domination.sum(0)
    
    Pf = []
    ranks = np.zeros(scores.shape[0])
    r = 0
    Q = np.argwhere(Nx == 0)
    while Q.size > 0:
        Q = Q[:, 0]
        Nx[Q] = -1
        Pf.append(Q)
        ranks[Q] = r
        r += 1
        for i in Q:
            Nx[domination[i, :]] -= 1
        Q = np.argwhere(Nx == 0)
        
    return Pf, ranks

def crowding_distance(scores):
    indices = np.argsort(scores, 0)
    sorted_scores = np.take_along_axis(scores, indices, 0)
    cd = np.zeros(scores.shape[0])
    for k in range(scores.shape[1]):
        cd[indices[[0, -1], k]] = np.inf
        cd[indices[1:-1, k]] += (sorted_scores[2:, k] - sorted_scores[:-2, k]) / (sorted_scores[-1, k] - sorted_scores[0, k])
    return cd

def random_population(d, n, x_min, x_max):
    return np.hstack([np.random.uniform(x_min, x_max, (n, d))])

def tournament_selection(ranks, dists, n):
    candidates = np.random.choice(n, (n, 2), replace=True)
    mask = np.where(
        ranks[candidates[:, 0]] == ranks[candidates[:, 1]],
        dists[candidates[:, 0]] > dists[candidates[:, 1]],
        ranks[candidates[:, 0]] < ranks[candidates[:, 1]]
    )
    result = candidates[:, 1]
    result[mask] = candidates[mask, 0]
    return result

def crossover(x, p, eta): # simulated binary crossover
    n, d = x.shape
    l = n // 2
    mask = np.random.random(l) <= p
    m = np.sum(mask)
    mi = np.random.random((m, d))
    beta = np.where(
        mi < 0.5,
        np.power(2*mi, 1. / (eta+1.)),
        np.power(1. / (2.*(1-mi)), 1. / (eta+1.))
    )
    c1 = x[:l, :].copy()
    c2 = x[l:, :].copy()
    c1[mask, :] = 0.5 * (1 + beta) * x[:l, :][mask, :] + 0.5 * (1 - beta) * x[l:, :][mask, :]
    c2[mask, :] = 0.5 * (1 + beta) * x[:l, :][mask, :] + 0.5 * (1 - beta) * x[l:, :][mask, :]
    return np.vstack([c1, c2])

def mutation(x, x_min, x_max, p, eta): # polynomial mutation
    n, d = x.shape
    mask = np.random.random(x.shape[0]) <= p
    m = np.sum(mask)
    mi = np.random.random((m, d))
    beta = np.where(
        mi < 0.5,
        np.power(2*mi, 1. / (eta+1.)) - 1.,
        1. - np.power(2.*(1-mi), 1. / (eta+1.))
    )
    y = x.copy()
    y[mask, :] = np.where(
        mi < 0.5,
        x[mask, :] + beta * (x[mask, :] - x_min),
        x[mask, :] + beta * (x_max - x[mask, :])
    )
    return y

def elitist_selection(fronts, dists, to_take):
    taken = []
    for front in fronts:
        if len(front) <= to_take:
            taken += list(front)
            if len(front) == to_take:
                break
            to_take -= len(front)
        else:
            indices = np.argsort(-dists[front])[:to_take]
            taken += list(indices)
            break
    return taken

def constraint_violation(constraints):
    n, d = constraints.shape
    sort_indices = np.argsort(constraints, 0)
    violations = np.zeros(n)
    for i in range(d):
        values, counts = np.unique(constraints[:, i], return_counts=True) # unikalne wartości są zwracane posortowane
        counts = np.cumsum(counts)
        counts = list(counts)
        if values[0] != 0:
            counts = [0] + counts
        for rank, (j, k) in enumerate(zip([0] + counts, counts + [len(counts)])):
            violations[sort_indices[j:k, i]] += rank
    return violations
        

def IDEA(objective, constraint, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=10):
    n_f = n - n_inf
    population = random_population(d, n, x_min, x_max)
    populations = [population.copy()]
    constraint_values = constraint(population)
    violation_measure = constraint_violation(constraint_values)
    scores = np.stack([objective(population), violation_measure], 1)
    scores_hist = [scores.copy()]

    fronts, ranks = FNS(scores)
    dists = crowding_distance(scores)
    
    for iter_ in range(num_iterations):
        parent_indices = tournament_selection(ranks, dists, n)
        offspring = crossover(population[parent_indices, :], p_c, eta_c)
        offspring = np.clip(offspring, x_min, x_max)
        offspring = mutation(offspring, x_min, x_max, p_m, eta_m)
        
        offspring_constraint_values = constraint(offspring)
        offspring_violation_measure = constraint_violation(offspring_constraint_values)
        offspring_scores = np.stack([objective(offspring), offspring_violation_measure], 1)
        
        population = np.vstack([population, offspring])
        scores = np.vstack([scores, offspring_scores])
        
        dists = crowding_distance(scores)
        mask_f = scores[:, -1] == 0
        mask_inf = ~mask_f
        s_f = np.sum(mask_f)
        s_inf = np.sum(mask_inf)
        if s_f < n_f:
            to_take_f = s_f
            to_take_inf = n - s_f
        elif s_inf < n_inf:
            to_take_inf = s_inf
            to_take_f = n - s_inf
        else:
            to_take_f = n_f
            to_take_inf = n_inf
            
        population_f = population[mask_f, :]
        scores_f = scores[mask_f, :]
        dists_f = dists[mask_f]
        fronts, ranks = FNS(population_f)
        taken_f = elitist_selection(fronts, dists_f, to_take_f)
        
        population_inf = population[mask_inf]
        scores_inf = scores[mask_inf, :]
        dists_inf = dists[mask_inf]
        fronts, ranks = FNS(population_inf)
        taken_inf = elitist_selection(fronts, dists_inf, to_take_inf)
        
        population = np.vstack([population_f[taken_f, :], population_inf[taken_inf, :]])
        scores = np.vstack([scores_f[taken_f, :], scores_inf[taken_inf, :]])
        dists = np.hstack([dists_f[taken_f], dists_inf[taken_inf]])
        fronts, ranks = FNS(population)
        
        populations.append(population.copy())
        scores_hist.append(scores.copy())
        
        if iter_ % log_interval == 0:
            print(f"Iteration {iter_}, #feasible: {to_take_f}, #infeasible: {to_take_inf}, scores: {scores.min(0)} {scores.mean(0)} {scores.max(0)}")
    print(f"Iteration {iter_}, #feasible: {to_take_f}, #infeasible: {to_take_inf}, scores: {scores.min(0)} {scores.mean(0)} {scores.max(0)}")
    return np.stack(populations, 0), np.stack(scores_hist, 0)

In [3]:
def g1_objective(x):
    return 5 * x[:, :4].sum(1) - 5 * (x[:, :4]**2).sum(1) - x[:, 4:].sum(1)

def g1_constraints(x):
    c = np.stack([
        2*x[:, 0] + 2*x[:, 1] + x[:, 9] + x[:, 10] - 10.,
        2*x[:, 0] + 2*x[:, 2] + x[:, 9] + x[:, 11] - 10.,
        2*x[:, 1] + 2*x[:, 2] + x[:, 10] + x[:, 11] - 10.,
        -8*x[:, 0] + x[:, 9],
        -8*x[:, 1] + x[:, 10],
        -8*x[:, 2] + x[:, 11],
        -2*x[:, 3] - x[:, 4] + x[:, 9],
        -2*x[:, 5] - x[:, 6] + x[:, 10],
        -2*x[:, 7] - x[:, 8] + x[:, 11],
    ], 1)
    c = np.maximum(c, 0.)
    return c
    
def g6_objective(x):
    return (x[:, 0] - 10.)**3 + (x[:, 1] - 20.)**3

def g6_constraints(x):
    c = np.stack([
        -(x[:, 0] - 5)**2 - (x[:, 1]-5)**2 + 100,
        (x[:, 0] - 6)**2 + (x[:, 1]-5)**2 - 82.81,
    ], 1)
    c = np.maximum(c, 0.)
    return c

def g2_objective(x):
    n, d = x.shape
    c = np.cos(x)
    return -np.abs(((c**4).sum(1) - 2*(c**2).prod(1)) / np.sqrt((np.arange(d) * x**2).sum(1)))

def g2_constraints(x):
    n, d = x.shape
    c = np.stack([
        0.75 - x.prod(1),
        x.sum(1) - 7.5*d
    ], 1)
    c = np.maximum(c, 0.)
    return c

In [7]:
objective = g6_objective
constraint = g6_constraints

x_min = np.array([13., 0.])
x_max = 100.
d = 2
n = 200
n_inf = int(0.2*n)
eta_c = 15.
eta_m = 20.
p_c = 0.8
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, constraint, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

Iteration 0, #feasible: 0, #infeasible: 200, scores: [-7.63417173e+03  1.00000000e+00] [1.24024143e+05 4.96900000e+01] [6.67001142e+05 1.87000000e+02]
Iteration 100, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.12527305e+03  5.20000000e+00] [-6785.07600771    41.        ]
Iteration 200, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.16135349e+03  5.58500000e+00] [-6958.30948156    42.        ]
Iteration 300, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.06621316e+03  3.88500000e+00] [681.00872736  35.        ]
Iteration 400, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.040777e+03  4.605000e+00] [346.95981677  40.        ]
Iteration 500, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.16276009e+03  5.42500000e+00] [-6960.18842353    42.        ]
Iteration 600, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.04371204e+03  4.42000000e+00] [-1731.6967357    43.       ]
Iteration 700, #feasible: 160, #inf

In [8]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-6961.636838707012

In [30]:
objective = g1_objective
constraint = g1_constraints

x_min = 0.
x_max = np.ones(13)
x_max[9:12] = 100.
d = 13
n = 600
n_inf = int(0.2*n)
eta_c = 5.
eta_m = 20.
p_c = 0.9
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, constraint, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

Iteration 0, #feasible: 0, #infeasible: 600, scores: [-289.34824275  149.        ] [-142.27481399 2275.94333333] [ -15.73844284 5140.        ]
Iteration 100, #feasible: 480, #infeasible: 120, scores: [-300.22359651    0.        ] [-33.04894525 161.63166667] [3.43711316e+00 3.51700000e+03]
Iteration 200, #feasible: 480, #infeasible: 120, scores: [-294.95056705    0.        ] [-28.55675298 168.91333333] [   4.45541191 3517.        ]
Iteration 300, #feasible: 480, #infeasible: 120, scores: [-300.898411    0.      ] [-24.1402392  120.62166667] [   4.7191875 3517.       ]
Iteration 400, #feasible: 480, #infeasible: 120, scores: [-300.41991953    0.        ] [-26.21705475 123.84      ] [   4.49105552 3517.        ]
Iteration 500, #feasible: 480, #infeasible: 120, scores: [-302.11293177    0.        ] [-28.3055452  138.88333333] [   4.57627684 3517.        ]
Iteration 600, #feasible: 480, #infeasible: 120, scores: [-302.41057669    0.        ] [-27.24021869 137.73      ] [   4.59994468 3517. 

In [31]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-8.262634449573463

In [32]:
objective = g2_objective
constraint = g2_constraints

x_min = 1e-4
x_max = 10.
d = 20
n = 400
n_inf = int(0.2*n)
eta_c = 4.
eta_m = 20.
p_c = 0.8
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, constraint, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

/home/michal/miniconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/home/michal/miniconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in greater


Iteration 0, #feasible: 398, #infeasible: 2, scores: [-0.1736712  0.       ] [-0.10740618  0.005     ] [-0.03310691  1.        ]
Iteration 100, #feasible: 320, #infeasible: 80, scores: [-0.37296823  0.        ] [-0.18694223 11.185     ] [-5.95436217e-03  1.05000000e+02]
Iteration 200, #feasible: 320, #infeasible: 80, scores: [-0.5368364  0.       ] [-0.25265908 11.2225    ] [-4.87545602e-03  1.08000000e+02]
Iteration 300, #feasible: 320, #infeasible: 80, scores: [-1.02553628  0.        ] [-0.33273886 11.595     ] [-4.39812243e-03  1.08000000e+02]
Iteration 400, #feasible: 320, #infeasible: 80, scores: [-1.67099871  0.        ] [-0.40340391 11.255     ] [-6.21749226e-03  1.08000000e+02]
Iteration 500, #feasible: 320, #infeasible: 80, scores: [-3.33480279  0.        ] [-0.56315028 11.535     ] [-9.2170283e-03  1.0800000e+02]
Iteration 600, #feasible: 320, #infeasible: 80, scores: [-5.20273421  0.        ] [-0.65476971  9.7875    ] [-6.05014779e-03  1.08000000e+02]
Iteration 700, #feasibl

In [33]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-0.8329168367456218

Sanity check

In [34]:
i = scores[-1, scores[-1, :, 1] == 0., 0].argmin()
i

3

In [35]:
x = populations[-1][scores[-1, :, 1] == 0., :][[i], :]

In [36]:
objective(x)

array([-0.83291684])

In [37]:
constraint(x)

array([[0., 0.]])